In [ ]:
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
from amftrack.pipeline.development.high_mag_videos.plot_data import (
    save_raw_data,
    plot_summary
)
# from amftrack.pipeline.development.high_mag_videos.high_mag_videos_fun import segment_brightfield
import os
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from tifffile import imwrite
from tifffile import imread
from tqdm import tqdm
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
from pathlib import Path
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300


In [ ]:
def segment_brightfield(image, thresh=0.5e-6, frangi_range=np.arange(70, 170, 30), seg_thresh = 11, binning=2):
    frangi_range = frangi_range * 2 / binning
    smooth_im_blur = cv2.blur(-image, (11, 11))
    smooth_im = frangi(-smooth_im_blur, frangi_range)
    smooth_im = np.array(smooth_im * (255/np.max(smooth_im)), dtype=np.uint8)
    seg_shape = smooth_im.shape

#     for i in range(1, 100):
#         _, segmented = cv2.threshold(smooth_im, i, 255, cv2.THRESH_BINARY)
#         coverage = 100 * np.sum(1 * segmented.flatten()) / (255 * seg_shape[0] * seg_shape[1])
#         if coverage < seg_thresh:
#             break
    return smooth_im
    

vid_folder = Path("/gpfs/scratch1/shared/amftrackflow/videos/CocoTransport/")
plate = "20230729_Plate440"
vid_interest = "090"
img_address = vid_folder / plate / vid_interest

imgs = sorted([path for path in img_address.glob("*/*.ti*")])
img = imread(imgs[0])

print(imgs[0])

In [ ]:
# segmented = segment_brightfield(img, frangi_range = np.arange(5, 40, 3))
segmented = segment_brightfield(img, frangi_range = np.arange(10, 150, 20))
_, segment_thresh = cv2.threshold(segmented, 0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
segment_thresh = cv2.morphologyEx(segment_thresh, cv2.MORPH_CLOSE, np.ones((250,250)))

fig, ax = plt.subplots(2, 2, figsize=(10, 5))
ax[0][0].imshow(img)
ax[0][1].imshow(segmented)
ax[1][0].imshow(segment_thresh)
fig.tight_layout()

# fig, ax = plt.subplots()
# ax.hist(segmented.flatten())